<a href="https://colab.research.google.com/github/zamoraricardo15/Intro-to-Health-Informatics/blob/main/Covid_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Approximation for misinformation estimation: COVID-19

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
import zipfile
from tensorflow.keras import Sequential, Input
from tensorflow.keras.utils import get_file
from sklearn.model_selection import train_test_split
BATCH_SIZE = 64

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [6]:
fake_df = pd.DataFrame(dict(title=[], isFakeNews=[], src=[]))

Fake and real news dataset:
https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

In [7]:
fakecsv_df = pd.read_csv('/content/Fake.csv')
truecsv_df = pd.read_csv('/content/True.csv')

fakecsv_df['isFakeNews'] = True
truecsv_df['isFakeNews'] = False
fakecsv_df['src'] = 'Fake-and-real-news-dataset'
truecsv_df['src'] = 'Fake-and-real-news-dataset'
fake_df = fake_df\
                    .append(fakecsv_df[['title', 'isFakeNews', 'src']])\
                    .append(truecsv_df[['title', 'isFakeNews', 'src']])

FakeNewsNet:
https://github.com/KaiDMML/FakeNewsNet

In [8]:
politic_df = pd.read_csv("/content/politifact_real.csv")
politifake_df = pd.read_csv("/content/politifact_fake.csv")
politic_df['isFakeNews'] = False
politifake_df['isFakeNews'] = True
politifact_df = politic_df.append(politifake_df)
politifact_df['src'] = 'FakeNewsNet/politifact'
fake_df = fake_df.append(politifact_df[['title', 'isFakeNews', 'src']])

In [9]:
realgossip_df = pd.read_csv("/content/gossipcop_real.csv")
fakegossip_df = pd.read_csv("/content/gossipcop_fake.csv")
realgossip_df['isFakeNews'] = False
fakegossip_df['isFakeNews'] = True
gossipcop_df = realgossip_df.append(fakegossip_df)
gossipcop_df['src'] = 'FakeNewsNet/gossipcop'
fake_df = fake_df.append(gossipcop_df[['title', 'isFakeNews', 'src']])

COVID-19 Rumor Dataset:
https://github.com/MickeysClubhouse/COVID-19-rumor-dataset

In [10]:
newss_df = pd.read_csv('/content/en_dup.csv')
newss_df = newss_df[~(newss_df['label'] == 'U')] 
newss_df['label'] = newss_df['label']  == 'F' 
newss_df['src'] = 'COVID-19-rumor-dataset' 
newss_df = newss_df[['label', 'content', 'src']].rename(columns={'label':'isFakeNews','content':'title'}) 
fake_df = fake_df.append(newss_df)

FakeCovid:
https://github.com/Gautamshahi/FakeCovid

In [11]:
jun_df = pd.read_csv("/content/FakeCovid_June2020.csv")
jul_df = pd.read_csv("/content/FakeCovid_July2020.csv")
jun_df.loc[jun_df['class'] == 'TRUE', 'isFakeNews'] = False
jul_df.loc[jul_df['class'] == 'TRUE', 'isFakeNews'] = False
jun_df['isFakeNews'].fillna(True, inplace = True)
jul_df['isFakeNews'].fillna(True, inplace = True)
jun_df['src'] = 'FakeCovid'
jul_df['src'] = 'FakeCovid'
fake_df = fake_df\
                    .append(jun_df[['ref_title', 'isFakeNews','src']].rename(columns={'ref_title':'title'}))\
                    .append(jul_df[['source_title', 'isFakeNews','src']].rename(columns={'source_title':'title'}))

In [12]:
fake_df.dropna(inplace=True)
fake_df.to_csv("data_fake_df.csv", index=False)

In [13]:
fake_df.head()

,title,isFakeNews,src
0,Donald Trump Sends Out Embarrassing New Year’...,1.0,Fake-and-real-news-dataset
1,Drunk Bragging Trump Staffer Started Russian ...,1.0,Fake-and-real-news-dataset
2,Sheriff David Clarke Becomes An Internet Joke...,1.0,Fake-and-real-news-dataset
3,Trump Is So Obsessed He Even Has Obama’s Name...,1.0,Fake-and-real-news-dataset
4,Pope Francis Just Called Out Donald Trump Dur...,1.0,Fake-and-real-news-dataset


In [14]:
df_data = "/content/data_fake_df.csv"
df = pd.read_csv(df_data)
df.head()

,title,isFakeNews,src
0,Donald Trump Sends Out Embarrassing New Year’...,1.0,Fake-and-real-news-dataset
1,Drunk Bragging Trump Staffer Started Russian ...,1.0,Fake-and-real-news-dataset
2,Sheriff David Clarke Becomes An Internet Joke...,1.0,Fake-and-real-news-dataset
3,Trump Is So Obsessed He Even Has Obama’s Name...,1.0,Fake-and-real-news-dataset
4,Pope Francis Just Called Out Donald Trump Dur...,1.0,Fake-and-real-news-dataset


In [15]:
lenmmax = df["title"].apply(lambda x : len(x.split())).max()
worcount = 100000
length_data = len(df)
length_data

86002

In [16]:
def train_test_split(dataset, length_data, val_split=0.2, shuffle=True, shuffle_size=50000):
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed=42)
    train_size = int((1-val_split) * length_data)
    val_size = int(val_split * length_data)
    try:
      train_ds = dataset.take(train_size).map(lambda x : (x["title"], x["isFakeNews"]))
      val_ds = dataset.skip(train_size).take(val_size).map(lambda x : (x["title"], x["isFakeNews"]))
    except:
      train_ds = dataset.take(train_size)
      val_ds = dataset.skip(train_size).take(val_size)
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return train_ds, val_ds

In [17]:
ds = tf.data.experimental.make_csv_dataset(df_data, select_columns=[
    "title",
    "isFakeNews"                                                                    
], batch_size=BATCH_SIZE)

train_ds, val_ds = train_test_split(ds, length_data)
val_ds, test_ds = train_test_split(val_ds, int(length_data * 0.2), val_split=0.5)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [18]:
def check_letters(text):
  marksp = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
  sowords = {'whom', 'all', 'shouldn', 'wouldn', 'how', 's', 'they', 'were', 'mustn', 'after', 'who', 'its', 'our', 't', 'a', 'very', 'an', 'do', 'be', 'to', 'can', 'had', 'i', 'these', 'himself', 'up', 'just', 'them', 'now', 'has', 'too', 'below', 'did', 'shan', 'until', 'during', 'him', 'into', 'have', "you'd", 'haven', 'theirs', 'ourselves', 'once', "isn't", 'than', "it's", 'wasn', 'yours', "mightn't", 'here', 'ours', 'her', 'doing', 'd', 'yourself', 'y', 'before', 'does', 'then', 'between', 'some', 'with', "needn't", 'but', 'didn', "shouldn't", 'that', "weren't", 'which', 'or', "hasn't", 'own', 'about', 'what', "aren't", 'couldn', 'doesn', 'as', "wouldn't", 'hasn', 'no', 'm', 'hers', 'hadn', 'aren', 'while', 'will', "don't", "shan't", 'why', 'at', 'mightn', 'themselves', 'weren', "that'll", 'isn', 'only', 'the', 'been', "couldn't", 'don', 'should', 'same', 'both', 'where', 'was', 'me', 'through', "hadn't", 've', 'against', 'if', 'under', 'such', 'is', 'll', "haven't", 'ain', 're', "didn't", 'nor', 'not', 'being', 'are', 'your', 'over', 'off', 'having', 'by', "won't", 'myself', 'out', 'more', "wasn't", "doesn't", 'won', 'this', 'my', 'again', 'ma', 'his', 'when', 'you', 'there', 'herself', 'yourselves', 'itself', 'of', "she's", 'needn', 'we', "mustn't", 'above', "you're", 'so', 'it', "should've", 'am', 'he', 'those', 'further', 'she', 'down', 'on', "you'll", 'for', 'other', 'any', 'their', 'from', 'each', 'most', 'because', 'and', 'few', 'in', "you've", 'o'}
  text = tf.strings.lower(text)
  text = tf.strings.strip(text)
  text = tf.strings.regex_replace(text, "<[^>]+>", "")
  text = tf.strings.regex_replace(text, '[%s]' % marksp, "")
  for soword in sowords:
    text = tf.strings.regex_replace(text, r"\b%s\b" % soword, "")
  return text
example1 = "Is COVID really happening and is it deadly?"
print(check_letters(example1))

tf.Tensor(b' covid really happening    deadly', shape=(), dtype=string)


In [19]:
def token_define(train_ds, worcount, lenmmax, output_mode = "int", standardize = "lower_and_strip_marksp"):
  train_text = train_ds.map(lambda x, y : x)
  ltokenl = tfl.TextVectorization(
      standardize=standardize,
      max_tokens=worcount,
      output_sequence_length=lenmmax,
      output_mode=output_mode
  )
  ltokenl.adapt(train_text)
  return ltokenl

In [20]:
def token_definition(train_ds, worcount, lenmmax=None, output_mode = "int", standardize = "lower_and_strip_marksp"):
  train_text = train_ds.map(lambda x, y : x)
  ltokenl = tfl.TextVectorization(
      standardize=standardize,
      max_tokens=worcount,
      output_mode=output_mode
  )
  ltokenl.adapt(train_text)
  return ltokenl

In [21]:
ltokenl = token_definition(train_ds, worcount, lenmmax, standardize=check_letters)

In [22]:
def check_pretrained_initial(url, output_file, embedding_file, embedding_dim, vocabulary, worcount, lenmmax):
  embedding_vecs = dict()
  word_idx = dict(zip(vocabulary, range(len(vocabulary))))
  file_dir = get_file(output_file, url)
  with zipfile.ZipFile(file_dir, "r") as f:
    f.extractall("/content/")
  with open(embedding_file, "r") as f:
    for line in f:
      values = line.split()
      word = values[0]
      embedding_vec = np.asarray(values[1:], dtype='float32')
      embedding_vecs[word] = embedding_vec
  embedding_matrix = np.zeros((worcount, embedding_dim))
  for word, idx in word_idx.items():
    if idx < worcount:
      embedding_vec = embedding_vecs.get(word)
      if embedding_vec is not None:
        embedding_matrix[idx] = embedding_vec
  
  embedding = tfl.Embedding(worcount, embedding_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), mask_zero=True, input_length=lenmmax, trainable=False)
  return embedding

In [23]:
def check_pretrained_after(url, output_file, embedding_file, embedding_dim, vocabulary, worcount, lenmmax=None):
  embedding_vecs = dict()
  word_idx = dict(zip(vocabulary, range(len(vocabulary))))
  file_dir = get_file(output_file, url)

  with zipfile.ZipFile(file_dir, "r") as f:
    f.extractall("/content/")

  with open(embedding_file, "r") as f:
    for line in f:
      values = line.split()
      word = values[0]
      embedding_vec = np.asarray(values[1:], dtype='float32')
      embedding_vecs[word] = embedding_vec

  embedding_matrix = np.zeros((worcount, embedding_dim))
  
  for word, idx in word_idx.items():
    if idx < worcount:
      embedding_vec = embedding_vecs.get(word)
      if embedding_vec is not None:
        embedding_matrix[idx] = embedding_vec
  
  embedding = tfl.Embedding(worcount, embedding_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), input_dim=len(vocabulary), trainable=False)
  return embedding

In [24]:
vocabulary = ltokenl.get_vocabulary()
embedding = check_pretrained_initial("https://nlp.stanford.edu/data/glove.6B.zip", "glove.6B.zip", "glove.6B.100d.txt", 100, vocabulary=vocabulary, worcount=worcount, lenmmax=lenmmax)


862182613/862182613 [==============================] - 160s 0us/step


In [25]:
def review_fake_check(ltokenl, embedding_layer, worcount, lenmmax, optimizer='adam'):
  model = Sequential(
      [
      ltokenl,
      embedding_layer,
      tfl.Bidirectional(tfl.LSTM(128, return_sequences=True, input_shape=(worcount, lenmmax))),
      tfl.Bidirectional(tfl.LSTM(128, return_sequences=False)),
      tfl.Dropout(0.2),
      tfl.Dense(128, activation='relu')
      tfl.Dense(64, activation='relu')
      tfl.Dense(1, activation='sigmoid')
      ]
  )
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['acc'])
  model.summary()
  return model

In [26]:
model = review_fake_check(ltokenl, embedding, worcount, lenmmax)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         10000000  
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        234496    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                        

In [27]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, TerminateOnNaN, EarlyStopping
checkpoint_path = "/content/"
callbacks = [
             TensorBoard(),
             ModelCheckpoint(checkpoint_path),
             ReduceLROnPlateau(),
             TerminateOnNaN(),
             EarlyStopping(patience=2)
]

In [28]:
def train_model(model, training_ds, validation_ds = None, val_split = 0.2, batch_size = BATCH_SIZE, epochs=5, callbacks=callbacks):
  if validation_ds is None:
    trainable_check = model.fit(training_ds, validation_split=val_split, batch_size=batch_size, epochs=epochs, callbacks=callbacks)
  else:
    trainable_check = model.fit(training_ds, validation_data=validation_ds, batch_size=batch_size, epochs=epochs, callbacks=callbacks)
  return trainable_check

In [ ]:
trainable_check = train_model(model, train_ds, val_ds, epochs=3)


Epoch 1/5
   8927/Unknown - 2634s 295ms/step - loss: 0.0099 - accuracy: 0.9961

In [ ]:
import tensorflowjs as tfjs
def model_to_tfhs(model, output_dir):
  tfjs.converters.save_keras_model(model, output_dir)

In [ ]:
predicted_result = model().predict(embedding)

prediction = []
for i in range (len(predicted_result)):
  if predicted_result[i].item() > 0.5:
    prediction.append(1)
  else:
    prediction.append(0)
    

#accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(list(test_ds), prediction)

print(accuracy)

In [ ]:
model.save("/content/model2")

In [ ]:
del model

In [ ]:
from tensorflow.keras.models import load_model
clone = load_model('/content/model1')

In [ ]:
!mv saved_model '/content/backup'